<a href="https://colab.research.google.com/github/karri-ten/VibeScope/blob/main/VibeScope.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VibeScope _ Analysing online vibe through NLP

VibeScope analyzes Twitter conversations in real time, using NLP and machine learning to extract and interpret sentiment trends. It provides insights on topics, brands, and events while leveraging automated data pipelines and interactive dashboards to enhance understanding of public opinion.

In [ ]:
# Install Tweepy (v4.10+ for API v2 Support)
!pip install tweepy matplotlib pandas numpy seaborn plotly

In [16]:
# Authenticate and Initialize Client
import tweepy
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.express as px
from google.colab import userdata

#from dotenv import load_dotenv
#load_dotenv()  # Load .env file

# Retrieve the BEARER_TOKEN from Google Colab secrets
bearer_token = userdata.get("BEARER_TOKEN")

# Check if the token was retrieved successfully
if bearer_token is None:
    raise ValueError("Google Colab secret 'BEARER_TOKEN' not found. Please ensure you have added it to Colab secrets.")

client = tweepy.Client(
    bearer_token=bearer_token, # Use the token retrieved from Colab secrets
    wait_on_rate_limit=True  # Pause if rate limit is hit
)

# check if all the credential are retrieve sucessfully

In [17]:
from google.colab import userdata
import tweepy
import os

# Retrieve all necessary credentials from Google Colab secrets
bearer_token = userdata.get("BEARER_TOKEN")
consumer_key = userdata.get("API_key")
consumer_secret = userdata.get("API_key_secret")
access_token = userdata.get("Access_Token")
access_token_secret = userdata.get("Access_Token_Secret")

# Check if all credentials were retrieved successfully
if any(token is None for token in [bearer_token, consumer_key, consumer_secret, access_token, access_token_secret]):
    raise ValueError("One or more Google Colab secrets (BEARER_TOKEN, CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET) not found. Please ensure you have added them to Colab secrets.")

# Initialize client with both Bearer Token and OAuth 1.0a credentials
client = tweepy.Client(
    bearer_token=bearer_token,
    consumer_key=consumer_key,
    consumer_secret=consumer_secret,
    access_token=access_token,
    access_token_secret=access_token_secret,
    wait_on_rate_limit=True  # Pause if rate limit is hit
)

# Now you can test both types of authentication
test_bearer = client.get_me()  # Uses Bearer Token by default
print("Test with Bearer Token:")
print(test_bearer)

test_user_auth = client.get_me(user_auth=True) # Uses OAuth 1.0a
print("\nTest with User Authentication (OAuth 1.0a):")
print(test_user_auth)

Test with Bearer Token:
Response(data=<User id=761580484128305154 name=Teniola Kareemat username=kareeprecious>, includes={}, errors=[], meta={})

Test with User Authentication (OAuth 1.0a):
Response(data=<User id=761580484128305154 name=Teniola Kareemat username=kareeprecious>, includes={}, errors=[], meta={})


In [ ]:
query = "Bitcoin -is:retweet lang:en"  # English, no retweets
tweets = []

# Use Paginator to handle rate limits
for tweet in tweepy.Paginator(
    client.search_recent_tweets,
    query=query,
    max_results=10,  # Max per request (free tier limit)
    limit=1  # 1 requests = 100 tweets
).flatten():  # Flatten paginated responses
    tweets.append(tweet.text)

print(f"Fetched {len(tweets)} tweets.")

In [ ]:
# Fetching of Tweets (Scrape 1000 Tweets About "ChatGPT" (Exclude Retweets))

query = "ChatGPT lang:en -is:retweet"
tweets = []

# Fetch tweets with metadata (e.g., likes, time)
tweets_data = []
for tweet in tweepy.Paginator(
    client.search_recent_tweets,
    query=query,
    max_results=100,
    tweet_fields=["created_at", "public_metrics", "author_id"],
    user_fields=["username"],
    expansions=["author_id"],
    limit=5
).flatten():
    tweets_data.append({
        "text": tweet.text,
        "created_at": tweet.created_at,
        "likes": tweet.public_metrics["like_count"],
        "retweets": tweet.public_metrics["retweet_count"],
        "author_id": tweet.author_id
    })

df = pd.DataFrame(tweets_data)
df.to_csv("tweets_chatgpt.csv", index=False)

KeyboardInterrupt: 

# Let try fix

In [19]:
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.2 MB/s eta 0:00:00


In [20]:
import ipywidgets as widgets
from IPython.display import display

# Create text input for search keyword
keyword_widget = widgets.Text(
    value='ChatGPT',
    placeholder='Type a keyword or hashtag',
    description='Keyword:',
    disabled=False
)

# Create slider or number input for tweet count
tweet_count_widget = widgets.IntSlider(
    value=100,
    min=10,
    max=1000,
    step=10,
    description='Num Tweets:',
    disabled=False
)

display(keyword_widget)
display(tweet_count_widget)

Text(value='ChatGPT', description='Keyword:', placeholder='Type a keyword or hashtag')

IntSlider(value=100, description='Num Tweets:', max=1000, min=10, step=10)

In [24]:
query = f"{keyword_widget.value} lang:en -is:retweet"
num_tweets = tweet_count_widget.value

tweets_data = []
for tweet in tweepy.Paginator(
    client.search_recent_tweets,
    query=query,
    max_results=100,
    tweet_fields=["created_at", "public_metrics", "author_id"],
    user_fields=["username"],
    expansions=["author_id"],
    limit=(num_tweets // 100)
).flatten(limit=num_tweets):
    tweets_data.append({
        "text": tweet.text,
        "created_at": tweet.created_at,
        "likes": tweet.public_metrics["like_count"],
        "retweets": tweet.public_metrics["retweet_count"],
        "author_id": tweet.author_id
    })

print(f"Fetched {len(tweets_data)} tweets.")

Fetched 0 tweets.
